In [ ]:
import os
from neo4j import GraphDatabase
from neo4j import exceptions
from pyprojroot import here
from dotenv import load_dotenv

from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI
from langchain_community.graphs import Neo4jGraph

In [6]:
# import environment variables
load_dotenv()
URI = os.environ["NEO4J_URI"]
AUTH = (os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"])
DATABASE = os.environ["NEO4J_DATABASE"]

In [7]:
graph = Neo4jGraph()
graph.refresh_schema()
print(graph.schema)

Node properties:
Movie {id: STRING, runtimeMinutes: STRING, releasedYear: DATE, title: STRING, averageRatings: FLOAT, numVotes: FLOAT}
Person {name: STRING, personId: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Movie)-[:OF_GENRE]->(:Genre)
(:Person)-[:ACTED_IN]->(:Movie)


In [8]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True, allow_dangerous_requests=True)
response = chain.invoke({"query": """Who are the actors of the three most popular genres of movies 
                         released in 1995 based on user ratings arrange them in descreasing order?"""})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)-[:OF_GENRE]->(g:Genre)
WHERE m.releasedYear = date("1995") 
RETURN p.name, g.name, m.averageRatings
ORDER BY m.averageRatings DESC
LIMIT 3
Full Context:
[{'p.name': 'Morgan Freeman', 'g.name': 'Drama', 'm.averageRatings': 8.6}, {'p.name': 'Brad Pitt', 'g.name': 'Drama', 'm.averageRatings': 8.6}, {'p.name': 'David Fincher', 'g.name': 'Drama', 'm.averageRatings': 8.6}]

> Finished chain.


{'query': 'Who are the actors of the three most popular genres of movies \n                         released in 1995 based on user ratings arrange them in descreasing order?',
 'result': 'Morgan Freeman, Brad Pitt, David Fincher are the actors of the three most popular genres of movies released in 1995 based on user ratings, arranged in decreasing order.'}

In [9]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True, allow_dangerous_requests=True)
response = chain.invoke({"query": """Name 3 most popular movies released in 1995 and their actors  based on user ratings arrange them in descreasing order?"""})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {releasedYear: date("1995-01-01")})
RETURN m.title, m.averageRatings
ORDER BY m.averageRatings DESC
LIMIT 3
Full Context:
[{'m.title': 'Se7en', 'm.averageRatings': 8.6}, {'m.title': 'The Usual Suspects', 'm.averageRatings': 8.5}, {'m.title': 'Braveheart', 'm.averageRatings': 8.3}]

> Finished chain.


{'query': 'Name 3 most popular movies released in 1995 and their actors  based on user ratings arrange them in descreasing order?',
 'result': 'Se7en - 8.6 (Brad Pitt, Morgan Freeman)\nThe Usual Suspects - 8.5 (Kevin Spacey, Gabriel Byrne)\nBraveheart - 8.3 (Mel Gibson, Sophie Marceau)'}

In [10]:
graph = Neo4jGraph()

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True, allow_dangerous_requests=True)
response = chain.invoke({"query": """Who are the actors of the three most popular genres of movies released in 1995 based on user ratings arrange them in descreasing order?"""})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:ACTED_IN]->(:Movie)-[:OF_GENRE]->(:Genre)
WHERE (:Movie {releasedYear: 1995})-[:OF_GENRE]->(:Genre)
RETURN p.name, COUNT(*) AS numMovies
ORDER BY numMovies DESC
LIMIT 3
Full Context:
[]

> Finished chain.


{'query': 'Who are the actors of the three most popular genres of movies released in 1995 based on user ratings arrange them in descreasing order?',
 'result': "I don't know the answer."}